In [1]:
import os
import sys
sys.path.append(os.path.abspath("/users/amtseng/tfmodisco/src/"))
sys.path.append(os.path.abspath("/users/amtseng/tfmodisco/notebooks/reports/"))
from util import figure_to_vdom_image
import plot.viz_sequence as viz_sequence
import numpy as np
import h5py
import matplotlib.pyplot as plt
import vdom.helpers as vdomh
from IPython.display import display

### Define constants and paths

In [2]:
# Define parameters/fetch arguments
motif_files = [
    os.path.join(
        "/users/amtseng/tfmodisco/results/reports/tfmodisco_results/cache/",
        "singletask_profile_finetune",
        "MAX_singletask_profile_finetune_fold1",
        "task_0",
        "MAX_singletask_profile_finetune_task0_fold1_count",
        "all_motifs.h5"
    )
]
group_names = [""]

print("Motif files: %s" % motif_files)
print("Group names: %s" % group_names)

Motif files: ['/users/amtseng/tfmodisco/results/reports/tfmodisco_results/cache/singletask_profile_finetune/MAX_singletask_profile_finetune_fold1/task_0/MAX_singletask_profile_finetune_task0_fold1_count/all_motifs.h5']
Group names: ['']


### Helper functions
For plotting and organizing things

In [3]:
def import_motifs(motif_files, group_names):
    """
    Imports a set of motifs from the saved HDF5 files.
    `group_names` is a list of group names, one for each motif file.
    Returns a list of motifs as L x 4 arrays, a parallel list of
    motif names, and a dictionary mapping group names to lists of
    motif names.
    """
    motifs, motif_names = [], []
    groups = {}
    for motif_file, stem in zip(motif_files, group_names):
        groups[stem] = []
        with h5py.File(motif_file, "r") as f:
            for key in f.keys():
                motif_name = "%s:%s" % (stem, key)
                motif_names.append(motif_name)
                motifs.append(f[key]["cwm_trimmed"][:])
                groups[stem].append(motif_name)
    return motifs, motif_names, groups

### Show motifs

In [4]:
motifs, motif_names, motif_groups = import_motifs(motif_files, group_names)

In [5]:
# Flip all motifs to be the purine-rich version
for i, motif in enumerate(motifs):
    if np.sum(motif[:, [0, 2]]) < 0.5 * np.sum(motif):
        motifs[i] = np.flip(motif)

In [6]:
# Show aggregated and constituent motifs for each cluster
colgroup = vdomh.colgroup(
    vdomh.col(style={"width": "5%"}),
    vdomh.col(style={"width": "95%"})
)

header = vdomh.thead(
    vdomh.tr(
        vdomh.th("Motif key", style={"text-align": "center"}),
        vdomh.th("CWM", style={"text-align": "center"})
    )
)

rows = []
for i, (motif_key, motif) in enumerate(zip(motif_names, motifs)):
    fig = viz_sequence.plot_weights(motif, figsize=(20, 4), return_fig=True)
    fig.tight_layout()
    rows.append(
        vdomh.tr(
            vdomh.td(motif_key),
            vdomh.td(figure_to_vdom_image(fig))
        )
    )

display(vdomh.table(colgroup, header, vdomh.tbody(*rows)))
plt.close("all")

<table><colgroup><col style="width: 5%"></col><col style="width: 95%"></col></colgroup><thead><tr><th style="text-align: center">Motif key</th><th style="text-align: center">CWM</th></tr></thead><tbody><tr><td>:0_0</td><td><div style="display: inline-block"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAABaAAAAEgCAYAAABCX9QDAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/d3fzzAAAACXBIWXMAAAsTAAALEwEAmpwYAACVwElEQVR4nOzdd5xkRbn/8U917smzOUdYMpINCEoSFAVzwAjmeI3XeBVBf%2BpFr4oZwzVcARFFQZIkJeecdlk2552dHDqe%2Bv1RfaZ7Zmd3p9P0hO97X%2BfVYfqcru7tma7z1FNPGWstIiIiIiIiIiIiIiKVFqh1A0RERERERERERERkclIAWkRERERERERERESqQgFoEREREREREREREakKBaBFREREREREREREpCoUgBYRERERERERERGRqlAAWkRERERERERERESqQgFoEREREREREREREakKBaBFREREREREREREpCoUgBYRERERERERERGRqlAAWkRERERERERERESqIlTrBpRrxowZdsmSJbVuhoiIiIiIiIiIiMiU8NBDD7VZa2eO5rETPgC9ZMkSHnzwwVo3Q0RERERERERERGRKMMasH%2B1jVYJDRERERERERERERKpCAWgRERERERERERERqQoFoEVERERERERERESkKhSAFhEREREREREREZGqUABaRERERERERERERKpCAWgRERERERERERERqQoFoEVERERERERERESkKhSAFhEREREREREREZGqUABaRERERERERERERKpCAWgRERERERERERERqQoFoEVERERERERERESkKhSAFhEREREREREREZGqUABaRERERERERERERKpCAWgRERERERERERERqQoFoEVERERERERERESkKhSAFhEREREREREREZGqUABaRERERERERERERKpCAWgRERERERERERERqQoFoEVERERERERERESkKqoSgDbGnGGMWWmMWW2M%2BcIIP/%2B0MeZpY8zjxphbjDGLC36WNcY8mtuurkb7RERERERERERERKT6QpU%2BoDEmCPwEOA3YBDxgjLnaWvt0wcMeAY6x1vYbYz4M/DfwltzPBqy1R1S6XSIiIiIiIiIiIiIytqqRAX0csNpau8ZamwIuB84ufIC19jZrbX/u5r3Agiq0Q0RERERERERERERqqBoB6PnAxoLbm3L37cl7gesLbseMMQ8aY%2B41xry2Cu0TERERERERERERkTFQ8RIcxTDGvAM4BnhZwd2LrbWbjTHLgFuNMU9Ya58ftt8HgA8ALFq0aMzaKyIiIiIiIiIiIiKjV40M6M3AwoLbC3L3DWGMORX4MnCWtTbp32%2Bt3Zy7XAP8Czhy%2BL7W2kustcdYa4%2BZOXNmZVsvIiIiIiIiIiIiIhVRjQD0A8D%2BxpilxpgI8Fbg6sIHGGOOBH6BCz7vKLi/1RgTzV2fARwPFC5eKCIiIiIiIiIiIiITRMVLcFhrM8aYjwE3AkHgN9bap4wxFwAPWmuvBi4CGoA/G2MANlhrzwIOAn5hjPFwwfFvW2sVgBYRERERERERERGZgIy1ttZtKMsxxxxjH3zwwVo3Q0RERERERERERGRKMMY8ZK09ZjSPrUYJDhERERERERERERERBaBFREREREREREREpDoUgBYRERERERERERGRqqj4IoQiIiIiIjLJWQvZAcgmwEuBCUEwBqE6MMpxEREREZE8BaBFRERERGTPsknY9QB0PQG7HoT2h6DnOfCSLvBsAmA9sFkwBuqXQMsRMOOF0Hyou4y01PhFiIiIiEitKAAtIiIiIiJDeVnYcRs8/7%2Bw6W8u0GzTLuu5kM0Ou40LTvc8B5v/DsG4y5KeeTwsfz8seA2E6sfqVYiIiIjIOKAAtIiIiIiIOH0b4KlvwfpLXZmNTC8uqlwCL%2BU2gO23uizqe9Mw9zQ45Csw47iKNVtERERExi8FoEVEREREprrEDnj8a7D2d%2BBlXLZzpWV63OXmf8C2m2HmCXD096H54Mo/l4iIiIiMG1ohRERERERkqkp1waNfhL8vhTX/60psVCP4PERuAcPtN8MNR8Mdb4bedVV%2BThERERGpFQWgRURERESmom23wNXLYOUPIdvvFhUcS9Zz9aE3XQXXHgwrf%2BTKfoiIiIjIpKIAtIiIiIjIVJJNwUOfgn%2B/BlLtuy8sONZsxrXh0S/CradBYmdt2yMiIiIiFaUAtIiIiIjIVNG9Cq5/Aay%2BpPaB5%2BGyfbDjDrjmANh6U61bIyIiIiIVogC0iIiIiMhUsPkfcP1RLgid7a91a0ZmU5DugNvPdosiqiSHiIiIyISnALSIiIiIyGS35vdw55tdljFerVuzb9kBeOa78MCHXa1oEREREZmwFIAWEREREZnMnv2%2BC%2BSOt5Ib%2B5Lth7V/gDvfAl661q0RERERkRIpAC0iIiIiMhlZC49%2BCR77yvgtubEv2X7Ych3c9krITLAAuoiIiIgACkCLiIiIiExOj3wWVl48cYPPvmw/tN0Nt5xc65aIiIiISAlCtW6AiIiIiIhU2KqfwnM/n/jBZ192ADofq3UrRERERKQECkCLiIiIiEwmW25w2c8Trebzvky21yMiIiIyRagEh4iIiIjIZNHxGNz5RgVrRURERGTcUABaRERERGQy6N8Et5wCmb5at0REREREZJAC0CIiIiIiE52XgX%2BdCemuWrdERERERGQIBaBFRERERCa6Jy6AnufBZmrdEhERERGRIbQIoYiIiIjIRNZ2Pzz7XdV9FhEREZFxSRnQIiIiIiITVboX7nidgs8iIiIiMm4pAC0iIiIiMlE98BFItte6FSIiIiIie6QAtIiIiIjIRLT5Otj4F/AStW6JiIiIiMgeVSUAbYw5wxiz0hiz2hjzhRF%2B/mljzNPGmMeNMbcYYxYX/Ozdxpjnctu7q9E%2BEREREZEJLZuA%2B86DbH%2BtWyIiIiIislcVD0AbY4LAT4BXAgcDbzPGHDzsYY8Ax1hrDweuBP47t%2B804GvAC4HjgK8ZY1or3UYRERERkQntyW9CuqfWrRARERER2adQFY55HLDaWrsGwBhzOXA28LT/AGvtbQWPvxd4R%2B766cBN1tr23L43AWcAl1WhnSIiIjLOWQvbtsHOnZBKQSYD4TBEIjBrltuMqXUrRcZY33p49nu1X3gwVA%2BRaRCMgpeGdDekOmrbJhEREREZd6oRgJ4PbCy4vQmX0bwn7wWu38u%2B8yvaOhERERmX%2Bvvhttvgrrvg0Udh5UrYvNkFmMNhd%2BkHm611AWmABQvgwAPhiCPghBPgZS%2BDaLRWr0JkDDz0SRfwHWvTj4UFr4MZL4bmgyHSCtkkYAEDgbArDdKzCnY9AFv%2BAVtvApsZ%2B7aKiIiIyLhRjQD0qBlj3gEcA7ysyP0%2BAHwAYNGiRVVomYiIiIyFDRvg2mvhssvg/vtd4LinxwWYCyX2ssba6tVuu%2B46aGhwgenjj4e3vQ1e9SqYO7e6r0FkTLXdD1tvHLugbt0i2P/DsOzdEGqAYMwFmn2F18FlQ08/FqYdDUveDibgFkpc9RNof3Bs2iwiIiIi40o1FiHcDCwsuL0gd98QxphTgS8DZ1lrk8Xsa629xFp7jLX2mJkzZ1as4SIiIlJ91sI//%2BmCxAcc